In [1]:
from google.colab import files
uploaded = files.upload()


Saving claims_5.csv to claims_5.csv


In [1]:
import pandas as pd
df = pd.read_csv("claims_final.csv", quotechar='"')
df

,ClaimID,ReportingDelay,AccidentHour,Age,WeeklyRate,Gender,MaritalStatus,HoursWorkedPerWeek,DependentChildren,DependentsOther,PartTimeFullTime,DaysWorkedPerWeek,DateOfAccident,ClaimDescription,ReportedDay,InitialCaseEstimate,Incurred,DetailedClaimDescription,SeverityLabel,InputText
0,1,15,11.0,28.0,500.00,M,S,44.0,0.0,0.0,F,5.0,5/10/2005,STRUCK SCAFFOLDING STRAIN LOWER BACK,3Thurs,9500.0,151.254501,I suffered a lower back injury when a scaffold...,Low,"I was born 28 years ago. Socially, I identify ..."
1,2,22,5.0,29.0,500.00,M,S,38.0,0.0,0.0,F,5.0,28/10/2003,STRUCK KNIFE LACERATED LEFT THUMB,2Wed,11500.0,442.125024,I suffered a laceration on my left thumb when ...,Medium,My age is 29. Being male has influenced my exp...
2,3,22,7.0,28.0,197.37,M,M,16.0,0.0,0.0,P,3.0,25/05/2004,SLIPPED AND HIT STRAINED LEFT SHOULDER INJURY ...,2Wed,8000.0,1494.490505,"I slipped and hit my left shoulder, resulting ...",High,I’ve lived for 28 years now. Gender: male. Rel...
3,4,15,12.0,40.0,0.00,M,M,0.0,1.0,0.0,F,5.0,21/07/1994,HIT FALLING DOOR LACERATION LEFT SHOULDER INJU...,4Fri,500.0,138.900000,I sustained an injury when the door unexpected...,Low,"Currently, I am 40. Socially, I identify as ma..."
4,5,38,12.0,22.0,435.70,M,M,38.0,0.0,0.0,F,5.0,9/06/1992,STRUCK FALLING OBJECT LACERATION RIGHT RING FI...,4Fri,320.0,296.160000,I filed a claim for an injury sustained when I...,Medium,I'm currently 22. My gender is male. I am marr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21680,21687,23,9.0,53.0,200.00,M,M,38.0,0.0,0.0,F,5.0,18/06/2000,STRUCK KNIFE CUT RIGHT HAND CUT,1Tues,2650.0,582.166718,I was injured when a knife was accidentally st...,Medium,I am 53 years old. I identify as male. I live ...
21681,21688,15,13.0,23.0,500.00,M,S,40.0,0.0,0.0,F,5.0,23/06/2004,STRUCK WITH HAMMER FRACTURE RIGHT MIDDLE FINGER,3Thurs,1000.0,908.635098,I suffered a fracture in my right middle finge...,Medium,I’ve reached the age of 23. Being male has inf...
21682,21689,109,12.0,19.0,200.00,M,S,38.0,0.0,0.0,F,5.0,6/01/1996,STRUCK METAL GRINDING FOREIGN BODY LEFT BODY L...,2Wed,814.0,60.250000,I suffered an injury when my finger was caught...,Low,I'm currently 19. Gender: male. I consider mys...
21683,21690,7,9.0,34.0,553.13,M,S,38.0,0.0,0.0,F,5.0,5/12/1996,STRUCK MOVING STEEL LACERATION LEFT KNEE LEFT ...,3Thurs,1500.0,761.625000,"I was struck by a moving steel object, resulti...",Medium,I’ve reached the age of 34. I live my life as ...


In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
# Encode target labels
label_encoder = LabelEncoder()
df["SeverityEncoded"] = label_encoder.fit_transform(df["SeverityLabel"])
num_labels = len(label_encoder.classes_)


In [3]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["InputText"].tolist(),
    df["SeverityEncoded"].tolist(),
    test_size=0.2,
    stratify=df["SeverityEncoded"],
    random_state=42,
    shuffle=True
)


In [4]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)


c:\Users\Aeiman Imtiaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch

class ClaimDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {"labels": torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

train_dataset = ClaimDataset(train_encodings, train_labels)
val_dataset = ClaimDataset(val_encodings, val_labels)


In [6]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from transformers import RobertaConfig, RobertaForSequenceClassification

# Include num_labels when creating the config
config = RobertaConfig.from_pretrained("roberta-base", num_labels=3, hidden_dropout_prob=0.3)

# Pass only the config to the model loader
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-severity-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,

    # Core training setup
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Simulate larger batch size
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Sequence length & scheduler
    lr_scheduler_type="cosine",
    warmup_steps=500,

    # Evaluation and saving
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # if using f1 metric
    greater_is_better=True
)


def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
)

trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Aeiman Imtiaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.012900,0.000052,1.000000,1.000000,1.000000,1.000000
2,0.000100,0.000020,1.000000,1.000000,1.000000,1.000000
3,0.000100,0.000017,1.000000,1.000000,1.000000,1.000000


c:\Users\Aeiman Imtiaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Aeiman Imtiaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3255, training_loss=0.09188699552232468, metrics={'train_runtime': 64443.964, 'train_samples_per_second': 0.808, 'train_steps_per_second': 0.051, 'total_flos': 3664074288235032.0, 'train_loss': 0.09188699552232468, 'epoch': 3.0})

In [7]:
# Create a directory to save the model
model_save_path = "roberta_severity_model"

# Save model and tokenizer
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"✅ RoBERTa model and tokenizer saved to: {model_save_path}")


✅ RoBERTa model and tokenizer saved to: roberta_severity_model


In [8]:
import pandas as pd
df = pd.read_csv("claims_final.csv", quotechar='"')
df

,ClaimID,ReportingDelay,AccidentHour,Age,WeeklyRate,Gender,MaritalStatus,HoursWorkedPerWeek,DependentChildren,DependentsOther,PartTimeFullTime,DaysWorkedPerWeek,DateOfAccident,ClaimDescription,ReportedDay,InitialCaseEstimate,Incurred,DetailedClaimDescription,SeverityLabel,InputText
0,1,15,11.0,28.0,500.00,M,S,44.0,0.0,0.0,F,5.0,5/10/2005,STRUCK SCAFFOLDING STRAIN LOWER BACK,3Thurs,9500.0,151.254501,I suffered a lower back injury when a scaffold...,Low,"I was born 28 years ago. Socially, I identify ..."
1,2,22,5.0,29.0,500.00,M,S,38.0,0.0,0.0,F,5.0,28/10/2003,STRUCK KNIFE LACERATED LEFT THUMB,2Wed,11500.0,442.125024,I suffered a laceration on my left thumb when ...,Medium,My age is 29. Being male has influenced my exp...
2,3,22,7.0,28.0,197.37,M,M,16.0,0.0,0.0,P,3.0,25/05/2004,SLIPPED AND HIT STRAINED LEFT SHOULDER INJURY ...,2Wed,8000.0,1494.490505,"I slipped and hit my left shoulder, resulting ...",High,I’ve lived for 28 years now. Gender: male. Rel...
3,4,15,12.0,40.0,0.00,M,M,0.0,1.0,0.0,F,5.0,21/07/1994,HIT FALLING DOOR LACERATION LEFT SHOULDER INJU...,4Fri,500.0,138.900000,I sustained an injury when the door unexpected...,Low,"Currently, I am 40. Socially, I identify as ma..."
4,5,38,12.0,22.0,435.70,M,M,38.0,0.0,0.0,F,5.0,9/06/1992,STRUCK FALLING OBJECT LACERATION RIGHT RING FI...,4Fri,320.0,296.160000,I filed a claim for an injury sustained when I...,Medium,I'm currently 22. My gender is male. I am marr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21680,21687,23,9.0,53.0,200.00,M,M,38.0,0.0,0.0,F,5.0,18/06/2000,STRUCK KNIFE CUT RIGHT HAND CUT,1Tues,2650.0,582.166718,I was injured when a knife was accidentally st...,Medium,I am 53 years old. I identify as male. I live ...
21681,21688,15,13.0,23.0,500.00,M,S,40.0,0.0,0.0,F,5.0,23/06/2004,STRUCK WITH HAMMER FRACTURE RIGHT MIDDLE FINGER,3Thurs,1000.0,908.635098,I suffered a fracture in my right middle finge...,Medium,I’ve reached the age of 23. Being male has inf...
21682,21689,109,12.0,19.0,200.00,M,S,38.0,0.0,0.0,F,5.0,6/01/1996,STRUCK METAL GRINDING FOREIGN BODY LEFT BODY L...,2Wed,814.0,60.250000,I suffered an injury when my finger was caught...,Low,I'm currently 19. Gender: male. I consider mys...
21683,21690,7,9.0,34.0,553.13,M,S,38.0,0.0,0.0,F,5.0,5/12/1996,STRUCK MOVING STEEL LACERATION LEFT KNEE LEFT ...,3Thurs,1500.0,761.625000,"I was struck by a moving steel object, resulti...",Medium,I’ve reached the age of 34. I live my life as ...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21685 entries, 0 to 21684
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClaimID                   21685 non-null  object 
 1   ReportingDelay            21685 non-null  int64  
 2   AccidentHour              21684 non-null  float64
 3   Age                       21684 non-null  float64
 4   WeeklyRate                21684 non-null  float64
 5   Gender                    21684 non-null  object 
 6   MaritalStatus             21662 non-null  object 
 7   HoursWorkedPerWeek        21684 non-null  float64
 8   DependentChildren         21684 non-null  float64
 9   DependentsOther           21684 non-null  float64
 10  PartTimeFullTime          21684 non-null  object 
 11  DaysWorkedPerWeek         21684 non-null  float64
 12  DateOfAccident            21684 non-null  object 
 13  ClaimDescription          21684 non-null  object 
 14  Report

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.pipeline import Pipeline

df = df.dropna(subset=["SeverityLabel"])

# Step 2: Encode target
label_encoder = LabelEncoder()
df["SeverityEncoded"] = label_encoder.fit_transform(df["SeverityLabel"])

# Step 3: Define existing safe features
safe_features = [
    "ReportingDelay", "AccidentHour", "Age", "WeeklyRate", "Gender",
    "MaritalStatus", "HoursWorkedPerWeek", "DependentChildren", "DependentsOther",
    "PartTimeFullTime", "DaysWorkedPerWeek"
]

# Step 4: Prepare X and y
X = df[safe_features]
X = pd.get_dummies(X, drop_first=True)  # Handle categorical features
y = df["SeverityEncoded"]
# Drop any rows in X or y that contain NaNs
combined = pd.concat([X, y], axis=1).dropna()
X = combined.drop("SeverityEncoded", axis=1)
y = combined["SeverityEncoded"]


# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Step 6: Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", C=1.0, random_state=42))
    ]),
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=1000, random_state=42))
    ])
}

# Step 7: Train and evaluate
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")

    print(f"\n🔹 {name}")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    results.append({
        "Model": name,
        "Accuracy": acc,
        "F1 Score (weighted)": f1
    })

# Step 8: Summary
results_df = pd.DataFrame(results)
print("\n📊 Model Performance Summary:")
print(results_df.sort_values("F1 Score (weighted)", ascending=False))



🔹 Random Forest
              precision    recall  f1-score   support

        High       0.44      0.53      0.48      1445
         Low       0.47      0.45      0.46      1446
      Medium       0.34      0.30      0.32      1446

    accuracy                           0.42      4337
   macro avg       0.42      0.42      0.42      4337
weighted avg       0.42      0.42      0.42      4337


🔹 SVM
              precision    recall  f1-score   support

        High       0.49      0.57      0.53      1445
         Low       0.53      0.51      0.52      1446
      Medium       0.38      0.32      0.35      1446

    accuracy                           0.47      4337
   macro avg       0.46      0.47      0.46      4337
weighted avg       0.46      0.47      0.46      4337


🔹 Logistic Regression
              precision    recall  f1-score   support

        High       0.49      0.60      0.54      1445
         Low       0.50      0.60      0.55      1446
      Medium       0.40     

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Create the model
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1
)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict
y_pred = xgb_model.predict(X_test)

# Evaluate
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")

print("🔹 XGBoost Classifier")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Summary
print("\n📊 XGBoost Performance:")
print(f"Accuracy: {acc:.4f}")
print(f"Weighted F1 Score: {f1:.4f}")


c:\Users\Aeiman Imtiaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:20:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 XGBoost Classifier
              precision    recall  f1-score   support

        High       0.49      0.58      0.53      1445
         Low       0.53      0.52      0.53      1446
      Medium       0.36      0.30      0.33      1446

    accuracy                           0.47      4337
   macro avg       0.46      0.47      0.46      4337
weighted avg       0.46      0.47      0.46      4337


📊 XGBoost Performance:
Accuracy: 0.4676
Weighted F1 Score: 0.4619
